In [1]:
!pip install -q transformers datasets matplotlib
!!pip install -q accelerate torchinfo huggingface_hub


[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip


["\x1bWARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager. It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv\x1b\x1b",
 '\x1b',
 '\x1b[\x1b\x1bnotice\x1b\x1b[1;39;49m]\x1b\x1b A new release of pip is available: \x1b\x1b23.0.1\x1b\x1b -> \x1b\x1b23.1.1\x1b',
 '\x1b[\x1b\x1bnotice\x1b\x1b[1;39;49m]\x1b\x1b To update, run: \x1b\x1bpython -m pip install --upgrade pip\x1b']

In [2]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | bash
!apt-get install git-lfs

Detected operating system as Ubuntu/focal.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.0.9
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (3.3.0).
0 upgraded, 0 newly installed, 0 to remove and 26 not upgraded.


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
import os
import requests
import tqdm
import json
import pandas as pd
import numpy as np
import zipfile
from collections import Counter
import torch 
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from transformers import PreTrainedTokenizerFast
from tqdm.notebook import tqdm
from transformers import T5ForConditionalGeneration, T5Config, AdamW
from tqdm.auto import tqdm
from accelerate import Accelerator

In [5]:
file_id = '1yXvDTRRxjCLyAf4icU1sEMKF0NpssCnC'
destination = 'human_proteins.tsv'
hf_name = "t5-small"
max_length = 512
tokenizer_folder = "PROTNAME_tok"
save_folder = "PROTNAME"
vocab_size=32

In [6]:
file_id = '1yXvDTRRxjCLyAf4icU1sEMKF0NpssCnC'
destination = 'human_proteins.tsv'

def format_size(value):
  if value >= 1024**3:
    return f"{round(value / 1024**3 , 3)} GB"
  elif value >= 1024**2:
    return f"{round(value / 1024**2 , 3)} MB"
  elif value >= 1024:
    return f"{round(value / 1024 , 3)} KB"
  return f"{value} B"

# Code taken from https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)
    params = { 'id' : id, 'confirm' : 1 }
    response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768
    
    pb = tqdm(response.iter_content(CHUNK_SIZE))
    b_total = 0
    with open(destination, "wb") as f:
        for i,chunk in enumerate(pb):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                b_total += len(chunk)

                if i % 1000 == 0:
                  pb.set_description(f"written : {format_size(b_total)}")
                i += 1
    print("\n")
    print(f"saved all the data to {destination}. total size : {format_size(os.stat(destination).st_size)}")


download_file_from_google_drive(file_id, destination)

0it [00:00, ?it/s]



saved all the data to human_proteins.tsv. total size : 12.9 MB


In [7]:
dataframe = pd.read_csv("human_proteins.tsv", sep="\t")

In [8]:
dataframe = dataframe[dataframe["Sequence"].str.len() <= 510]

In [9]:
dataframe["Protein Name"] = [v.split("(")[0] for v in dataframe["Protein Name"]]

In [10]:
dataframe

,Sequence,Protein Name
0,MRWQEMGYIFYPRKLR,Mitochondrial-derived peptide MOTS-c
4,MPGWFKKAWYGLASLLSFSSFILIIVALVVPHWLSGKILCQTGVDL...,Clarin-2
6,MKRRQKRKHLENEESQETAEKGGGMSKSQEDALQPGSTRVAKGWSQ...,Protein FAM170A
7,MDDADPEERNYDNMLKMLSDLNKDLEKLLEEMEKISVQATWMAYDM...,Synaptonemal complex central element protein 3
8,MAFSDLTSRTVHLYDNWIKDADPRVEDWLLMSSPLPQTILLGFYVY...,Elongation of very long chain fatty acids prot...
...,...,...
20417,MPLASPIQHHEVTRGVAPSMALRDGVCRIPLSAEFTAQCTDSQAPQ...,Putative uncharacterized protein PRO3102
20418,MVRPHLLKKKILGRVWWLMPVVLALWEAEVGGSLEVRSLRPAWPTW,Putative uncharacterized protein PRO2829
20419,MAETYRRSRQHEQLPGQRHMDLLTGYSKLIQSRLKLLLHLGSQPPV...,Putative uncharacterized protein DKFZp434L187
20420,MAHHSLNTFYIWHNNVLHTHLVFFLPHLLNQPFSRGSFLIWLLLCW...,Putative uncharacterized protein encoded by LI...


In [11]:
with open("Sequences.txt", "w") as f:
    f.write("\n".join(dataframe["Sequence"].values))

In [12]:
with open("names.txt", "w") as f:
    f.write("\n".join([v.split("(")[0].strip() for v in dataframe["Protein Name"].values]))

In [13]:
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

# Train the input tokenizer 
input_tokenizer = ByteLevelBPETokenizer()
input_tokenizer.train(files=["Sequences.txt"], vocab_size=261, 
                      special_tokens=["<pad>", "<s>", "</s>","<unk>"])

input_tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", input_tokenizer.token_to_id("</s>")),
    ("<s>", input_tokenizer.token_to_id("<s>")),
)

# save the input tokenizer 
os.makedirs("input_tokenizer_aminoacids", exist_ok=True)
input_tokenizer.save_model("input_tokenizer_aminoacids")

['input_tokenizer_aminoacids/vocab.json',
 'input_tokenizer_aminoacids/merges.txt']

In [14]:
len(input_tokenizer.encode("MRWQEMGYIFYPRKLR"))

18

In [15]:
# Train the output tokenizer 
from tokenizers import SentencePieceBPETokenizer

output_tokenizer = SentencePieceBPETokenizer()
output_tokenizer.train(files=["names.txt"], vocab_size=32000,min_frequency=2, special_tokens=["<pad>","<s>","</s>","<unk>"])

# save the input tokenizer 
os.makedirs("output_tokenizer_names", exist_ok=True)
input_tokenizer.save_model("output_tokenizer_names")

['output_tokenizer_names/vocab.json', 'output_tokenizer_names/merges.txt']

In [16]:
e = output_tokenizer.encode("Protein FAM170A")

In [17]:
e.tokens

['▁Protein', '▁FAM17', '0', 'A']

In [18]:
_tokenized_X = []
_tokenized_Y = []

tokenized_X = []
tokenized_Y = []
sizes_X = []
sizes_Y = []

max_length_inp = 512
max_length_out = 32 

input_tok = PreTrainedTokenizerFast(tokenizer_object=input_tokenizer,
                                                   max_length=512,
                                                   padding=True)

output_tok = PreTrainedTokenizerFast(tokenizer_object=output_tokenizer,
                                                   max_length=32,
                                                   padding=True)

for x in tqdm(dataframe["Sequence"]):
    input_tokens = input_tok.encode(x)
    tokenized_X.append(input_tokens)
    sizes_X.append(len(input_tokens))

for y in tqdm(dataframe["Protein Name"]):
    out_tokens = output_tok.encode(y)
    tokenized_Y.append(out_tokens)
    sizes_Y.append(len(out_tokens))
    
for x,y in tqdm(zip(_tokenized_X, _tokenized_Y)):
   if len(x) <= max_length_inp and len(y) <= max_length_out:
    tokenized_X.append(x)
    tokenized_Y.append(y)

  0%|          | 0/12617 [00:00<?, ?it/s]

  0%|          | 0/12617 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [19]:
class CustomAADataset(Dataset):
  def __init__(self, max_length=512, max_length2=32):
    self.max_length = max_length
    self.max_length2 = max_length2 
    
  def __len__(self):
    return len(tokenized_X)

  def __getitem__(self, idx):
    input_tokens = tokenized_X[idx]
    output_tokens = tokenized_Y[idx]

    # Pad input and output tokens
    input_tokens = input_tokens + [input_tokenizer.token_to_id("<pad>")] * (self.max_length - len(input_tokens))

    output_tokens = output_tokens + [output_tokenizer.token_to_id("<pad>")] * (self.max_length2 - len(output_tokens))

    return {"input_ids": torch.tensor(input_tokens, dtype=torch.long),
            "labels": torch.tensor(output_tokens, dtype=torch.long),
            }


In [20]:
# Load data
from datasets import Dataset as Dataset_hf

dataset = Dataset_hf.from_dict({"source": dataframe["Sequence"].values, "target": dataframe["Protein Name"].values})

In [21]:
# Create dataset instances
train_dataset = CustomAADataset(max_length_inp, max_length_out)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [22]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

data_collator = DataCollatorForSeq2Seq(tokenizer=None, padding=True, max_length=512)

# Load T5 model
config = T5Config.from_pretrained("mlewand/PROT5-small")
model = T5ForConditionalGeneration(config)
#model.to(device)

In [28]:
training_args = Seq2SeqTrainingArguments(
    output_dir="PROT5-small",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    num_train_epochs=10,
    logging_steps = 250,
    fp16=True,  # Enable mixed precision training if your GPU supports it
    push_to_hub=True,
    hub_model_id="mlewand/PROT5-small"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=None,
)

trainer.train()

/workspace/PROT5-small is already a clone of https://huggingface.co/mlewand/PROT5-small. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
250,0.331400
500,0.346700
750,0.347200
1000,0.329600
1250,0.336500
1500,0.336300
1750,0.329100
2000,0.322900
2250,0.329300
2500,0.319700


TrainOutput(global_step=7890, training_loss=0.306198959930768, metrics={'train_runtime': 1106.3744, 'train_samples_per_second': 114.039, 'train_steps_per_second': 7.131, 'total_flos': 1.707607509172224e+16, 'train_loss': 0.306198959930768, 'epoch': 10.0})

In [35]:
batch = next(iter(train_loader))
input_ids = batch["input_ids"].to("cuda")
labels = batch["labels"].to("cuda")

In [36]:
outputs = model(input_ids=input_ids, labels=labels)

In [43]:
preds = torch.argmax(outputs.logits, dim=2)

In [60]:

for v in range(len(labels)):
    y_true = labels[v][labels[v] != 0]
    y_hat  = preds[v][preds[v] != 0]
    print(output_tokenizer.decode(y_true.cpu().detach().numpy()), output_tokenizer.decode(y_hat.cpu().detach().numpy()))

Beta-1,4-galactosyltransferase 5  Protein 5 
Gamma-aminobutyric acid receptor subunit gamma-1  Gamma-aminobutyric acid receptor subunit rho 
General transcription factor IIE subunit 1  Keratin, transcription factor IIH subunit 1 
Dehydrogenase/reductase SDR family member 7  Pregnancy-specific SDR family member 7 
WAP four-disulfide core domain protein 13 Putative four-disulfide core domain protein 3
Tripartite motif-containing protein 73  E3 motif-containing protein 74 
Sex comb on midleg-like protein 4 Zinc comb on midleg-like protein 4
Neuromedin-B [Cleaved into: Neuromedin-B-32; Neuromedin-B] Putative [Cleaved into: Neuromedin-B-32; Neuromedin-B]
Sperm protein associated with the nucleus on the X chromosome N5  Endogenous protein associated with the nucleus on the X chromosome D5 
Eukaryotic translation initiation factor 1  Histone translation initiation factor 5A 
Carcinoembryonic antigen-related cell adhesion molecule 7  Protein antigen-related cell adhesion molecule 8 
Interferon

'Transmembrane protein 140'

In [ ]:
"""
accelerator = Accelerator(mixed_precision="bf16")
device = accelerator.device

# Load T5 model
config = T5Config.from_pretrained("t5-small")
model = T5ForConditionalGeneration(config)
model.to(device)

# Set hyperparameters
num_epochs = 3
learning_rate = 5e-4

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate)
model, optimizer, data = accelerator.prepare(model, optimizer, train_loader)

log_every = 100
# Training loop
for epoch in range(num_epochs):
    print(f"Epoch: {epoch + 1}/{num_epochs}")

    # Training
    model.train()
    train_losses = []
    loss_total = 0
    samples_seen_so_far = 0
    
    pb = tqdm(train_loader)
    for i,batch in enumerate(pb):
        input_ids = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss
        
        loss_total += loss.item() * input_ids.shape[0]
        samples_seen_so_far += input_ids.shape[0]
        
        # backward pass
        accelerator.backward(loss)
        optimizer.step()
        
        if i % log_every == 0:
            pb.set_description(f"Loss : {round(loss_total / samples_seen_so_far, 4)}")

    print(f"Train loss: {loss_total / samples_seen_so_far}")
 """